### Pinecone Vector DB



In [1]:
## Create your index and apikey from here https://www.pinecone.io/
import os
# from dotenv import load_dotenv # Not needed when using userdata.get
# load_dotenv() # Not needed when using userdata.get
# api_key=os.getenv('PINECONE_API_KEY') # Not needed when using userdata.get

from google.colab import userdata
api_key = userdata.get('PINECONE_API_KEY')

In [2]:
!pip show langchain-pinecone

Name: langchain-pinecone
Version: 0.0.1
Summary: An integration package connecting Pinecone and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, numpy, pinecone-client, simsimd
Required-by: 


In [3]:
!pip install -qU langchain langchain-pinecone langchain-openai

In [4]:
from pinecone import Pinecone
pc=Pinecone(api_key=api_key)

In [5]:
from langchain_openai import OpenAIEmbeddings

# Assuming you have OPENAI_API_KEY stored in Colab Secrets Manager as well
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024,api_key=userdata.get('OPENAI_API_KEY'))

In [6]:
from pinecone import ServerlessSpec

index_name = "rag1"  # change if desired

try:
    pc.describe_index(index_name)
    print(f"Index '{index_name}' already exists.")
except:
    print(f"Index '{index_name}' does not exist. Creating index...")
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

Index 'rag1' already exists.


In [7]:
index

In [14]:
from langchain_pinecone import Pinecone

vector_store = Pinecone(index=index, embedding=embeddings)


In [15]:
vector_store

In [16]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [17]:
vector_store.add_documents(documents=documents)

['57bd880b-4d1d-467f-937d-cf6db0b755a0',
 'c8a1f470-7e83-48a9-a664-89ea3f014b87',
 '31bf7a05-905f-4888-aaff-e167c6837215',
 '43779fca-95f9-4b7a-8aa5-fe9e6db42199',
 '7d8d6627-dd1a-4a27-a33b-70f8b0f4e65a',
 '31f72804-c1eb-47ae-9b95-2b3ebe055376',
 'c93840c2-cbba-4c47-bf23-0ab9382f7e8d',
 '6547d13d-5c19-47cf-9563-528e592a3421',
 '221d48ff-9230-4a97-aa4b-be1b163f3fd1',
 '6b06c87f-70c1-4a05-b3cf-1df2f2c05675']

In [18]:
### Query Directly
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [19]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.572732] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [20]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})

[Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]